In [3]:
# importing dataframe.xlsx file

import pandas as pd
#df = pd.read_excel("dataframe_engabstract.xlsx")
df = pd.read_excel("dataframe_gerabstract.xlsx")
df.head()



,author,title,affiliation,email,GermanAbstract,year,journal,doi,street,city,zip
0,"Michael L. Brodie, Michael Stonebraker",DARWIN: On the Incremental Migration of Legacy...,Hochschule Albstadt-Sigmaringen,michael_l._brodie@hochschule_albstadt_sigmarin...,befragungen von mitarbeitern der verwaltung ha...,1997,ZA-Information / Zentralarchiv für Empirische ...,https://doi.org/10.1080/00036840701736115,Schoenebergerstrasse 79,Stralsund,18412
1,Frank Manola,Object Model Capabilities For Distributed Obje...,Fachhochschule für Verwaltung und Dienstleistu...,frank_manola@fachhochschule_für_verwaltung_und...,Acht Jahre nach der telefonischen Befragung vo...,2009,ZUMA Nachrichten,https://doi.org/10.1080/00036840701736115,Oldesloer Strasse 32,Suhl,98501
2,Frank Manola,Integrating Object-Oriented Applications and M...,Ostbayerische Technische Hochschule Amberg-Weiden,frank_manola@ostbayerische_technische_hochschu...,Das International Social Survey Programme (ISS...,2009,ZUMA Nachrichten,https://doi.org/10.1080/00036840701736115,Albrechtstrasse 84,Nußdorf,83131
3,"Frank Manola, Sandra Heiler",A RISC Object Model for Object System Interope...,Technische Hochschule Aschaffenburg,frank_manola@technische_hochschule_aschaffenbu...,Das GESIS-IZ Sozialwissenschaften hat im Auftr...,2008,ZUMA Nachrichten,https://doi.org/10.1080/00036840701736115,Gruenauer Strasse 59,Buxtehude,21614
4,Frank Manola,MetaObject Protocol Concepts for a RISC Object...,Fachhochschule Polizei Sachsen-Anhalt (Aschers...,frank_manola@fachhochschule_polizei_sachsen_an...,Die Studie präsentiert die Ergebnisse einer Be...,2005,ZUMA Nachrichten,https://doi.org/10.1080/00036840701736115,Prager Str 23,Weßling,82231


In [4]:
def generateAnnotationCsv(replaces, filelocation, template):

    # labels
    authorLabel = 1
    titleLabel = 2
    affilationLabel = 3
    dateLabel = 4
    journalLabel = 5
    addressLabel = 6
    abstractLabel = 7
    abstractLabel = 7
    doiLabel = 8
    emailLabel = 9
    otherLabel = 10

    annotation = []
    # Creating csv based on the order and length of replaces in template
    # TODO: Other Label if there is unknown amount of text left
    if template == '6041.docx':
        print("Template " + template + " found")
        authorLen = len(replaces[0].split())
        titleLen = len(replaces[1].split())
        gerintroductionLen = len(replaces[7].split())

        annotations = authorLen * [authorLabel] + titleLen * [titleLabel] + gerintroductionLen * [abstractLabel]
        #Rest is other label

    elif template == '11258.docx':
         print("Template " + template + " found")
         journalLen = len(replaces[4].split())
         dateLen = len(replaces[3].split())
         titleLen = len(replaces[1].split())
         authorLen = len(replaces[0].split())
         affilationLen = len(replaces[2].split())
         addressLen = len(replaces[5].split())

         annotations = journalLen * [journalLabel] + dateLen * [dateLabel] + titleLen * [titleLabel] + authorLen * [authorLabel] + affilationLen * [affilationLabel] + addressLen * [addressLabel]

    elif template == '11721.docx':
        print("Template " + template + " found")
        journalLen = len(replaces[4].split())
        authorLen = len(replaces[0].split())
        addressLen = len(replaces[5].split())
        doiLen = len(replaces[8].split())

        annotations = journalLen * [journalLabel] + 4 * [otherLabel] + authorLen * [authorLabel] + 25 * [otherLabel] + addressLen * [addressLabel] + 10 * [otherLabel] + doiLen * [doiLabel]
        
    else:
        print("No information about this template")


    if len(annotations) > 0:
        df = pd.DataFrame(annotations, columns=['Label'])
    else:
        df = pd.DataFrame([], columns=['Label'])

    df.to_csv(filelocation)


## Generating Data in form of .docx files 

In [5]:
# generating new data from template in the form of docx files 
import re
from docx import Document
from docx2pdf import convert  ##only works on Windows
import os

temp = ""
# function to replace text in docx file
def docx_replace_regex(doc_obj, regex , replace, temp):   
    # p variable refers to paragraph below
    for p in doc_obj.paragraphs:
        inline = p.runs     
        if regex in p.text:            
            for i in range(len(inline)):                
                temp = temp + inline[i].text
                if regex in temp:
                    if replace=="na":
                        continue 
                    else:
                        temp = temp.replace(regex, replace)
                        inline[i].text=temp        

# print function to check the layout of word file 
def printf (doc_obj):  
    for p in doc_obj.paragraphs:
        if p.text=='':
            continue
        else:
            print(p.text)
    print("\n")

# put the text you want to replace here, depending on how many classes of metadata is in that template

text_to_replace1 = "author"
text_to_replace2 = "title"
text_to_replace3 = "affiliation"
text_to_replace4 = "date"
text_to_replace5 = "journal"
text_to_replace6 = "address"
text_to_replace7 = "engintroduction"
text_to_replace8 = "gerintroduction"
text_to_replace9 = "doi"
text_to_replace10 = "email"


# select data from dataframe to replace with
count=0
number=0
stopnumber=0

templates_folder_name = 'templates/'

for file in os.scandir(templates_folder_name):
    if file.is_file():
        for row in df.index:
            replace1 = df['author'][count]
            replace2 = df['title'][count]
            replace3 = df['affiliation'][count]
            replace4 = str(df['year'][count])
            replace5 = df['journal'][count]
            replace6 = df['street'][count]
            try:
              replace7 = df['EnglishAbstract'][count]
            except KeyError:
              replace7 = ''  
            try:
              replace8 = df['GermanAbstract'][count]
            except KeyError:
              replace8 = ''

            replace9 = df['doi'][count]
            replace10 = df['email'][count]

            count=count+1

            ############### put the template docx file name here to read  ########################
            filename = file.name
            doc = Document("templates/" + filename)
            docx_replace_regex(doc, text_to_replace1 , replace1, temp)
            docx_replace_regex(doc, text_to_replace2 , replace2, temp)
            docx_replace_regex(doc, text_to_replace3 , replace3, temp)
            docx_replace_regex(doc, text_to_replace4 , replace4, temp)
            docx_replace_regex(doc, text_to_replace5 , replace5, temp)
            docx_replace_regex(doc, text_to_replace6 , replace6, temp)
            docx_replace_regex(doc, text_to_replace7 , replace7, temp)
            docx_replace_regex(doc, text_to_replace8 , replace8, temp)
            docx_replace_regex(doc, text_to_replace9 , replace9, temp)
            docx_replace_regex(doc, text_to_replace10 , replace10, temp)

            # TODO: Randomize some of the tempaltes e.g. pageNumbers, articleNumber in Journal, Address of publisher etc.
            #printf (doc)
            #print(" \n ---------------------------------------------------------------------------------------- \n")

            # save the generated file
            doc_filename = filename.split('.')[0] + '_' + str(number) +  '.docx'
            pdf_filename = filename.split('.')[0] + '_' + str(number) +  '.pdf'
            csv_filename = filename.split('.')[0] + '_' + str(number) +  '.csv'
            folder = 'generated_docs/'
            pdffolder = 'generated_pdfs/'
            annotationfolder = 'generated_annotations/'
            doc.save(folder+doc_filename)
            number = number+1
            stopnumber=stopnumber+1

            #convert(folder + doc_filename, pdffolder + pdf_filename) #(this can be enabled on Windows to generate the pdf too)


            generateAnnotationCsv([replace1, replace2, replace3, replace4, replace5, replace6, replace7, replace8, replace9, replace10], annotationfolder + csv_filename,filename)


            # put the stop number to how many files you want to generate
            if stopnumber==5:
                break


            #convert(folder + doc_filename, pdffolder + pdf_filename) #(this can be enabled on Windows to generate the pdf too)
            #oowriter -convert-to pdf:writer_pdf_Export *.docx (linux alternative command)

Template 11258.docx found


NotImplementedError: docx2pdf is not implemented for linux as it requires Microsoft Word to be installed

## Save only the first page

In [6]:
from PyPDF2 import PdfFileReader, PdfFileWriter
import os

#Save only the first page    
current_path = os.getcwd()
print(current_path)
dataset_folder_name = current_path + '/generated_pdfs/'
result_folder = current_path + '/generated_pdfs/'

for file in os.scandir(dataset_folder_name):
    if (file.path.endswith('.pdf')) and file.is_file():

        pdf = PdfFileReader(file.path)
        pdfWriter = PdfFileWriter()
        pdfWriter.addPage(pdf.getPage(0))
        new_path = result_folder + file.name
        print(new_path)
        f = open(result_folder +  file.name, 'wb')
        pdfWriter.write(f)
        f.close()

/home/azeddine/Dropbox/University/Winter semester 20-21/Machine learning application/research-lab-ml/data_generation
/home/azeddine/Dropbox/University/Winter semester 20-21/Machine learning application/research-lab-ml/data_generation/generated_pdfs/11721_1208.pdf
/home/azeddine/Dropbox/University/Winter semester 20-21/Machine learning application/research-lab-ml/data_generation/generated_pdfs/11721_1052.pdf
/home/azeddine/Dropbox/University/Winter semester 20-21/Machine learning application/research-lab-ml/data_generation/generated_pdfs/11721_1188.pdf
/home/azeddine/Dropbox/University/Winter semester 20-21/Machine learning application/research-lab-ml/data_generation/generated_pdfs/11721_1094.pdf
/home/azeddine/Dropbox/University/Winter semester 20-21/Machine learning application/research-lab-ml/data_generation/generated_pdfs/11721_1048.pdf
/home/azeddine/Dropbox/University/Winter semester 20-21/Machine learning application/research-lab-ml/data_generation/generated_pdfs/11721_1023.pdf
/